In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Unequal Weighting
loss_function = IntermediateScaledLoss(omega = 0.25)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-.25/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.042960980539520584
Average test loss: 0.011390960566699505
Epoch 2/300
Average training loss: 0.015603592342800563
Average test loss: 0.009589469021807115
Epoch 3/300
Average training loss: 0.013865230699380239
Average test loss: 0.009184941369626257
Epoch 4/300
Average training loss: 0.012974060365723239
Average test loss: 0.008502535868436098
Epoch 5/300
Average training loss: 0.012336067604521911
Average test loss: 0.008531822625133726
Epoch 6/300
Average training loss: 0.011760513368580076
Average test loss: 0.008800127218580908
Epoch 7/300
Average training loss: 0.011184872746467591
Average test loss: 0.007853030977149804
Epoch 8/300
Average training loss: 0.010775478868848748
Average test loss: 0.007616426826351219
Epoch 9/300
Average training loss: 0.010497133646574286
Average test loss: 0.0076063157659437925
Epoch 10/300
Average training loss: 0.01029369692173269
Average test loss: 0.00745219069553746

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 17.79
Average PSNR for Projection Layer 1 across 2500 images: 22.60
Average PSNR for Projection Layer 2 across 2500 images: 23.95
Average PSNR for Projection Layer 3 across 2500 images: 24.91
Average PSNR for Projection Layer 4 across 2500 images: 25.76
Average PSNR for Projection Layer 0 across 2500 images: 19.92
Average PSNR for Projection Layer 1 across 2500 images: 24.51
Average PSNR for Projection Layer 2 across 2500 images: 26.44
Average PSNR for Projection Layer 3 across 2500 images: 27.67
Average PSNR for Projection Layer 4 across 2500 images: 28.66
Average PSNR for Projection Layer 0 across 2500 images: 21.24
Average PSNR for Projection Layer 1 across 2500 images: 25.97
Average PSNR for Projection Layer 2 across 2500 images: 27.98
Average PSNR for Projection Layer 3 across 2500 images: 29.45
Average PSNR for Projection Layer 4 across 2500 images: 30.51
Average PSNR for Projection Layer 0 across 2500 images: 22.10
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-.25/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.10960823508765963
Average test loss: 0.034345837516917124
Epoch 2/300
Average training loss: 0.019873274137576422
Average test loss: 0.026894217398431567
Epoch 3/300
Average training loss: 0.0169536135925187
Average test loss: 0.010305846619937154
Epoch 4/300
Average training loss: 0.015408242400321695
Average test loss: 0.009779982426514228
Epoch 5/300
Average training loss: 0.014335512669550047
Average test loss: 0.028426721677184106
Epoch 6/300
Average training loss: 0.013501109267274538
Average test loss: 0.015517374361554781
Epoch 7/300
Average training loss: 0.01267437117960718
Average test loss: 0.00861976852433549
Epoch 8/300
Average training loss: 0.01203853105339739
Average test loss: 0.008379941106670433
Epoch 9/300
Average training loss: 0.011442527485390505
Average test loss: 0.007679658475021521
Epoch 10/300
Average training loss: 0.01083047271023194
Average test loss: 0.007442921287069718
Epoch

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 9.69
Average PSNR for Projection Layer 1 across 2500 images: 14.31
Average PSNR for Projection Layer 2 across 2500 images: 17.99
Average PSNR for Projection Layer 3 across 2500 images: 12.86
Average PSNR for Projection Layer 4 across 2500 images: 19.62
Average PSNR for Projection Layer 5 across 2500 images: 20.76
Average PSNR for Projection Layer 6 across 2500 images: 11.37
Average PSNR for Projection Layer 7 across 2500 images: 21.54
Average PSNR for Projection Layer 8 across 2500 images: 22.97
Average PSNR for Projection Layer 9 across 2500 images: 17.32
Average PSNR for Projection Layer 10 across 2500 images: 24.15
Average PSNR for Projection Layer 11 across 2500 images: 23.66
Average PSNR for Projection Layer 12 across 2500 images: 25.70
Average PSNR for Projection Layer 13 across 2500 images: 26.42
Average PSNR for Projection Layer 14 across 2500 images: 26.79
Average PSNR for Projection Layer 0 across 2500 images: 12.67
Aver

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-.25/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.30490926141209074
Average test loss: 0.01574680593940947
Epoch 2/300
Average training loss: 0.023643638971779082
Average test loss: 0.0601314547508955
Epoch 3/300
Average training loss: 0.01837852949400743
Average test loss: 0.010766306705772877
Epoch 4/300
Average training loss: 0.01584512079589897
Average test loss: 0.02519891009728114
Epoch 5/300
Average training loss: 0.01464501158396403
Average test loss: 0.009821665460036861
Epoch 6/300
Average training loss: 0.012840660291413467
Average test loss: 0.06966327463255988
Epoch 8/300
Average training loss: 0.01214609988944398
Average test loss: 0.0570890254245864
Epoch 9/300
Average training loss: 0.011609838649630546
Average test loss: 0.008955068062163061
Epoch 10/300
Average training loss: 0.011033663280308246
Average test loss: 0.007624070418377717
Epoch 11/300
Average training loss: 0.010653545477324063
Average test loss: 0.007634703187892835
Epoch 12/

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 4.04
Average PSNR for Projection Layer 1 across 2500 images: 3.69
Average PSNR for Projection Layer 2 across 2500 images: 7.19
Average PSNR for Projection Layer 3 across 2500 images: 9.05
Average PSNR for Projection Layer 4 across 2500 images: 5.96
Average PSNR for Projection Layer 5 across 2500 images: 9.35
Average PSNR for Projection Layer 6 across 2500 images: 8.84
Average PSNR for Projection Layer 7 across 2500 images: 11.05
Average PSNR for Projection Layer 8 across 2500 images: 12.27
Average PSNR for Projection Layer 9 across 2500 images: 12.45
Average PSNR for Projection Layer 10 across 2500 images: 17.63
Average PSNR for Projection Layer 11 across 2500 images: 13.88
Average PSNR for Projection Layer 12 across 2500 images: 20.38
Average PSNR for Projection Layer 13 across 2500 images: 9.04
Average PSNR for Projection Layer 14 across 2500 images: 17.40
Average PSNR for Projection Layer 15 across 2500 images: 22.31
Average PS